# Grid Search with G = 0.001 - 0.1 and n in 10k from 100k and d from 5-15

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from Data_for_bp import data_gather_from_files,run_strategy_optimised,run_strategy_eval,get_tick_data
import time

## Grid search

In [33]:
def optimize_gridsearch(tick_data):
    # For G: Values between 0.001 and 0.01 with an interval of 0.001
    G_values = np.arange(0.001, 0.011, 0.001)
    
    # For n: Values from 100000 to 5000000 with an interval of 10_000
    n_values = np.arange(100_000, 5_000_001, 10_000)

    d_values = np.arange(5, 16, 1)
    
    best_profit = float('-inf')
    best_G = None
    best_n = None
    best_d = None

    # Grid Search
    for G in G_values:
        for n in n_values:
            for d in d_values:
                max_loss, U_PNL, max_position, R_PNL = run_strategy_optimised(tick_data, G, n, d)
            
                constraint1_okay = (max_position <= 10e6)
                constraint2_okay = (U_PNL >= -150e3)
                constraint3_okay = (max_loss >= -500e3)

                if constraint1_okay and constraint2_okay and constraint3_okay and R_PNL > best_profit:
                    best_profit = R_PNL
                    best_G = G
                    best_n = n
                    best_d = d

    result = {
        "best_G": best_G,
        "best_n": best_n,
        "best_d": best_d,
        "best_profit": best_profit,
    }  
    return result

### test G,n,d optimize

In [35]:
months_data = data_gather_from_files('1 jan 2021','1 feb 2021')['EURUSD.mid']
test_data = data_gather_from_files('1 feb 2021','1 mar 2021')['EURUSD.mid']
time1 = time.time()
result = optimize_gridsearch(months_data)
time2 = time.time()
if result["best_G"] is not None:
        print("Optimization from 3 previous months")
        print(f"Optimal G: {result['best_G']:.3f}")
        print(f"Optimal n: {result['best_n']}")
        print(f"Optimal d: {result['best_d']}")
        print(f"Optimal net profit: {result['best_profit']}")
        print(f"Computing time:{time2-time1}")
        maximum_loss, min_unrealized_PNL, max_pos, realized_pnl=run_strategy_optimised(months_data,result['best_G'],result['best_n'],result['best_d'])
        print(f"maximum loss: {maximum_loss}")
        print(f"min unrealized PNL: {min_unrealized_PNL}")
        print(f"maximum position: {max_pos}")
        print(f"Realized profit: {realized_pnl}\n")
else:
        print("No feasible solution found")
maximum_loss, min_unrealized_PNL, max_pos, realized_pnl=run_strategy_optimised(test_data,result['best_G'],result['best_n'],result['best_d'])
print("test data results")
print(f"maximum loss: {maximum_loss}")
print(f"min unrealized PNL: {min_unrealized_PNL}")
print(f"maximum position: {max_pos}")
print(f"Realized profit: {realized_pnl}\n")

Optimization from 3 previous months
Optimal G: 0.001
Optimal n: 1350000
Optimal d: 6
Optimal net profit: 130544.9989
Computing time:3313.6476123332977
maximum loss: -25650.0
min unrealized PNL: -45457.491
maximum position: 9995400.0
Realized profit: 130544.9989

test data results
maximum loss: -102600.0
min unrealized PNL: -55125.0
maximum position: 9914400.0
Realized profit: -76328.9999



## Grid Search 3 months, test for 1 months

### Test

In [25]:
months_data = data_gather_from_files('1 jan 2021','1 mar 2021')['EURUSD.mid']
test_data = data_gather_from_files('1 mar 2021','1 may 2021')['EURUSD.mid']
time1 = time.time()
result = optimize_gridsearch(months_data)
time2 = time.time()
if result["best_G"] is not None:
        print("Optimization from 3 previous months")
        print(f"Optimal G: {result['best_G']:.3f}")
        print(f"Optimal n: {result['best_n']}")
        print(f"Optimal net profit: {result['best_profit']}")
        print(f"Computing time:{time2-time1}")
        maximum_loss, min_unrealized_PNL, max_pos, realized_pnl=run_strategy_optimised(months_data,result['best_G'],result['best_n'])
        print(f"maximum loss: {maximum_loss}")
        print(f"min unrealized PNL: {min_unrealized_PNL}")
        print(f"maximum position: {max_pos}")
        print(f"Realized profit: {realized_pnl}\n")
else:
        print("No feasible solution found")
maximum_loss, min_unrealized_PNL, max_pos, realized_pnl=run_strategy_optimised(test_data,result['best_G'],result['best_n'])
print("test data results")
print(f"maximum loss: {maximum_loss}")
print(f"min unrealized PNL: {min_unrealized_PNL}")
print(f"maximum position: {max_pos}")
print(f"Realized profit: {realized_pnl}\n")


Optimization from 3 previous months
Optimal G: 0.009000000000000001
Optimal n: 2780000
Optimal net profit: 175140.0
Computing time:375.8592472076416
maximum loss: 0
min unrealized PNL: -56295.0
maximum position: 9982980.0
Realized profit: 175140.0

test data results
maximum loss: -125100.0
min unrealized PNL: -125100.0
maximum position: 13110480.0
Realized profit: 56295.0



## Loop optimize from Jan to Dec, quarterly test

In [36]:
def run_optimization_and_test(start_date_str, end_date_str, test_end_date_str):
    months_data = data_gather_from_files(start_date_str, end_date_str)['EURUSD.mid']
    test_data = data_gather_from_files(end_date_str, test_end_date_str)['EURUSD.mid']
    
    time1 = time.time()
    result = optimize_gridsearch(months_data)
    time2 = time.time()
    
    if result["best_G"] is not None:
        print(f"Optimization from {start_date_str} to {end_date_str}")
        print(f"Optimal G: {result['best_G']:.3f}")
        print(f"Optimal n: {result['best_n']}")
        print(f"Optimal d: {result['best_d']}")
        print(f"Optimal net profit: {result['best_profit']}")
        print(f"Computing time:{time2-time1}")
        maximum_loss, min_unrealized_PNL, max_pos, realized_pnl = run_strategy_optimised(
            months_data, result['best_G'], result['best_n'], result['best_d']
        )
        print(f"maximum loss: {maximum_loss}")
        print(f"min unrealized PNL: {min_unrealized_PNL}")
        print(f"maximum position: {max_pos}")
        print(f"Realized profit: {realized_pnl}\n")
    else:
        print("No feasible solution found")
    
    maximum_loss, min_unrealized_PNL, max_pos, realized_pnl = run_strategy_optimised(
        test_data, result['best_G'], result['best_n'], result['best_d']
    )
    print("Test data results")
    print(f"maximum loss: {maximum_loss}")
    print(f"min unrealized PNL: {min_unrealized_PNL}")
    print(f"maximum position: {max_pos}")
    print(f"Realized profit: {realized_pnl}\n")

In [42]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Initial date
start_date = datetime(2021, 1, 1)

for _ in range(4):  # There are four quarters in a year
    end_date = start_date + relativedelta(months=3)  # Move to the start of the next quarter
    test_end_date = end_date + relativedelta(months=1)  # Move to the start of the next month
    
    # Convert datetime objects to string in 'DD MMM YYYY' format
    run_optimization_and_test(
        start_date.strftime('%d %b %Y'), 
        end_date.strftime('%d %b %Y'), 
        test_end_date.strftime('%d %b %Y')
    )
    
    start_date = end_date  # Move to the next quarter


KeyboardInterrupt: 

In [41]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Initial date
start_date = datetime(2021, 1, 1)

for _ in range(4):  # There are four quarters in a year
    end_date = start_date + relativedelta(months=3)  # Move to the start of the next quarter
    test_end_date = end_date + relativedelta(months=1)  # Move to the start of the next month

    months_data = data_gather_from_files(start_date.strftime('%d %b %Y'), end_date.strftime('%d %b %Y'),)['EURUSD.mid']
    test_data = data_gather_from_files(end_date.strftime('%d %b %Y'),test_end_date.strftime('%d %b %Y'))['EURUSD.mid']
    
    print(months_data)
    print("------------------")
    print(test_data)



    start_date = end_date  # Move to the next quarter


Date
2021-01-03 22:00:00.040000+00:00    1.224035
2021-01-03 22:00:00.144000+00:00    1.224025
2021-01-03 22:00:00.434000+00:00    1.224020
2021-01-03 22:00:00.738000+00:00    1.224015
2021-01-03 22:00:00.933000+00:00    1.224005
                                      ...   
2021-03-31 23:59:11.418000+00:00    1.172740
2021-03-31 23:59:22.791000+00:00    1.172735
2021-03-31 23:59:23.103000+00:00    1.172740
2021-03-31 23:59:36.258000+00:00    1.172745
2021-03-31 23:59:49.514000+00:00    1.172720
Name: EURUSD.mid, Length: 5402382, dtype: float32
------------------
Date
2021-04-01 00:00:00.135000+00:00    1.172725
2021-04-01 00:00:00.490000+00:00    1.172710
2021-04-01 00:00:00.591000+00:00    1.172705
2021-04-01 00:00:00.794000+00:00    1.172700
2021-04-01 00:00:02.097000+00:00    1.172695
                                      ...   
2021-04-30 20:59:56.818000+00:00    1.202065
2021-04-30 20:59:57.673000+00:00    1.202075
2021-04-30 20:59:58.170000+00:00    1.202080
2021-04-30 20:59:59.0